In [2]:
import calendar
import json
from collections import Counter
from datetime import date
from pprint import pprint
import os
from pymongo import MongoClient
import pandas as pd

In [5]:
data = pd.read_json('sl_self_serv_corrections.json', orient='records',lines=True)
data["label"].value_counts().head(50)

AttributeError: 'list' object has no attribute 'value_counts'

In [6]:
ticketlist = []
output = []

with open('sl_sentiment_predictions.json') as file:
	for obj in file:
		ticket = json.loads(obj)
		ticketlist.append(ticket)

In [7]:
for ticket in ticketlist:
    output.append(ticket['ticket_id'])

In [8]:
with open(os.path.expanduser('~/.mongo/mongo.json'),'r') as config:
    mongo_uri = json.load(config)['mongo_uri']
client = MongoClient(mongo_uri)
db = client.forethought

organization_id = '429'
start = '2021-06-11T00:00:00Z'
end = '2021-06-25T00:00:00Z'
mongo_org_collection = db[str(organization_id)]

In [ ]:
def get_dataset():
    tickets = mongo_org_collection.find({
        'doc_type': 'ticket',
        "is_deleted": False,
        'original_item.via.channel': {'$in':['web','email', "api"]},
        'original_item.created_at': {'$gte': start, '$lte': end},
        'original_item.custom_fields': {"$ne":
            {
                "id" : 360044502352,
                "value" : None
            }
        }
    })
    
    file_path = os.path.expanduser('~/Downloads/sl_sentiment_predictions.json')
    with open(file_path, 'w') as f:
        count = 0
        for ticket in tickets:
            i = ticket["original_item"]
            j = {
                "ticket_id": i['id'], 
                # "subject": i["subject"],
                # "description": i["description"],
                # "created_at": i["created_at"],
                # "label": [x['value'] for x in i['custom_fields'] if x['id'] == 360044502352]
                }
                
            json.dump(j, f)
            _ = f.write('\n')
            count += 1
            if count % 100 == 0:
                print(count)

    print("total: ", count)


get_dataset()

In [ ]:
with open("sluv_sentiment.json", "w") as f:
    for ticket in output:
        json.dump(ticket, f)
        f.write("\n")